### Generates the predictions to be submitted to kaggle

1. Generate embeddings for all users from training data
2. Use model to make predictions for users who have purchases >0
3. For users that dont have purchases, predict most popular from last week

In [1]:
import numba

In [2]:
import pandas as pd, os, numpy as np
import plotly.express as px
pd.options.display.max_columns = 50
import swifter, datetime, pickle as pkl
import tensorflow_hub as hub
from tqdm.notebook import tqdm

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import regularizers

In [4]:
with open('./data/all_cemb_map.pkl', 'rb') as handle:
    all_cemb_map = pkl.load(handle)

In [5]:
with open('./data/scaler.pkl', 'rb') as handle:
    scaler = pkl.load(handle)

In [17]:
keys = list(all_cemb_map.keys())

In [18]:
values = list(all_cemb_map.values())

In [23]:
model = keras.models.load_model('./models/model1')

In [24]:
values_fs = np.array(values)

In [25]:
values_fs.shape

(1362281, 512)

In [26]:
values_fs = scaler.transform(values_fs)

In [27]:
preds = model.predict(values_fs, verbose=1)

42572/42572 [==============================] - 50s 1ms/step


In [28]:
preds.shape

(1362281, 512)

In [29]:
df = pd.read_parquet('./data/train.parquet')

In [30]:
d_end = datetime.datetime(2020, 9, 22).date()
d_start = datetime.datetime(2020, 9, 1).date()

In [31]:
tdf = df[(df['date'] >= d_start) & (df['date'] <= d_end)].reset_index(drop=True).copy()

In [32]:
uaids = tdf['article_id'].unique()

In [33]:
with open('./data/emb_map.pkl', 'rb') as handle:
    emb_map = pkl.load(handle)

In [34]:
f_emb_map = {}
for aid in uaids:
    f_emb_map[aid] = emb_map[aid]

In [35]:
len(f_emb_map)

26252

In [36]:
len(uaids)

26252

In [37]:
akeys = list(f_emb_map.keys())
aembs = list(f_emb_map.values())

In [38]:
aembs = np.array(aembs)

In [39]:
akeys = np.array(akeys)

In [40]:
aembsT = aembs.T

In [51]:
# def calculateSimilarity(queryVector):
#     scores = queryVector.dot(aembsT).flatten() # cosine similarity scores. (assumes vectors are normalized to unit length)
#     return scores

def calculateSimilarity2(queryVector):
    scores = np.linalg.norm(aembs - queryVector, axis=1).flatten()
    return scores

In [52]:
aembs.shape

(26252, 512)

In [53]:
def rankPostsBySimilarity(posts_ids, scores):
    return sorted([(post_id, score) for (post_id, score) in zip(posts_ids, scores)], key=lambda x:x[1], reverse=False)[:12]

In [54]:
import time

In [55]:
preds[0][:10]

array([-0.04759209,  0.04022962, -0.02809096, -0.00954398,  0.02968043,
        0.02860958,  0.03878016,  0.0705606 ,  0.06684647, -0.01625933],
      dtype=float32)

In [56]:
preds[1][:10]

array([-0.04577065,  0.03328498, -0.03254869, -0.01129808,  0.03612462,
        0.03349651,  0.04484773,  0.07012564,  0.06950598, -0.02301978],
      dtype=float32)

In [57]:
cpreds = []
start=time.time()
for p in tqdm(preds[:10], total=len(preds)):
    sims = calculateSimilarity2(p)
    temp = np.argpartition(sims, -12)[-12:]
    r = rankPostsBySimilarity(akeys[temp], sims[temp])
    rank_ids = [v[0] for v in r]
    
    preds_sep_f = ''
    for i in range(len(rank_ids)):
        val = '0' + str(rank_ids[i])
        if i != len(rank_ids) - 1:
            val += ' '
        preds_sep_f += val
    cpreds.append(preds_sep_f)
end = time.time()
print("total time taken: {}".format(end-start))
    # rank_ids = [v[0] for v in r]
    
    # preds_sep_f = ''
    # for i in range(len(rank_ids)):
    #     val = '0' + str(rank_ids[i])
    #     if i != len(rank_ids) - 1:
    #         val += ' '
    #     preds_sep_f += val
    # cpreds.append(preds_sep_f)

  0%|          | 0/1362281 [00:00<?, ?it/s]

total time taken: 0.4391510486602783


In [49]:
adf = pd.read_parquet('./data/articles.parquet')

In [61]:
adf[adf['article_id'] == 809590003]

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_no,department_name,index_name,index_group_name,section_no,section_name,garment_group_name,detail_desc
82708,809590003,809590,Headphones,78,Other accessories,Accessories,Solid,Turquoise,Medium,Turquoise,4313,Girls Small Acc/Bags,"Children Accessories, Swimwear",Baby/Children,43,"Kids Accessories, Swimwear & D",Accessories,"Adjustable, shimmering on-ear headphones with ..."


In [ ]:
pdf = pd.DataFrame()
pdf['cust_id'] = keys
pdf['prediction'] = cpreds

In [ ]:
pdf.to_parquet('./data/cpreds.parquet')

In [ ]:
pdf